<a href="https://colab.research.google.com/github/SKam23/10315-Final-Project/blob/main/10315_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-315 ML Final Project
Created by Shaheer Aslam, Steven Kam, and Sajan Shah.

< explanation of project here >

## Library Imports

In [ ]:
import importlib
import torch 
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from PIL import Image

if importlib.util.find_spec("kaggle") is None:
    !pip install -q kaggle

# CelebA Dataset Processing

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d mariafrenti/celeba

403 - Forbidden


In [ ]:
!unzip -n celeba-dataset.zip

Archive:  celeba-dataset.zip


In [ ]:
# list_eval_partition.csv: Recommended partitioning of images into training, validation, testing sets. Images 1-162770 are training, 162771-182637 are validation, 182638-202599 are testing
# list_bbox_celeba.csv: Bounding box information for each image. "x_1" and "y_1" represent the upper left point coordinate of bounding box. "width" and "height" represent the width and height of bounding box
# list_landmarks_align_celeba.csv: Image landmarks and their respective coordinates. There are 5 landmarks: left eye, right eye, nose, left mouth, right mouth
# list_attr_celeba.csv: Attribute labels for each image. There are 40 attributes. "1" represents positive while "-1" represents negative
partition_df = pd.read_csv('list_eval_partition.csv')
bbox_df = pd.read_csv('list_bbox_celeba.csv')
landmarks_df = pd.read_csv('list_landmarks_align_celeba.csv')
attr_df = pd.read_csv('list_attr_celeba.csv')



In [ ]:
merged_df = partition_df.merge(bbox_df, on='image_id').merge(landmarks_df, on='image_id').merge(attr_df, on='image_id')
train_df = merged_df[merged_df['partition'] == 0]
val_df = merged_df[merged_df['partition'] == 1]
test_df = merged_df[merged_df['partition'] == 2]


In [ ]:
class CelebADataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.transform = ToTensor()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(f"celeba/img_align_celeba/{row['image_id']}")
        image = self.transform(image)
        is_young = row["young"] == 1
        label = 1 if is_young else 0
        return image, label


In [ ]:
train_dataset = CelebADataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CelebADataset(val_df)
val_dataloader = DataLoader(val_dataset, batch_size=64)

test_dataset = CelebADataset(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=64)

# Convolutional Neural Network
< explanation >

In [ ]:
# We create a PyTorch CNN from scratch, using two convolutional layers and one fully connected layer (for now).
class CNN(nn.Module):
  def __init__(self):
    # supering nn.Module
    super(CNN, self).__init__()

    # creating first convolutional layer
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    # creating second convolutional layer
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    # creating third convolutional layer
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    # creating fully connected layers
    self.fc1 = nn.Linear(64 * 8 * 8, 512)
    self.fc2 = nn.Linear(512, 128)
    self.fc3 = nn.Linear(128, 40)
  
  # feed forward portion of CNN
  def forward(self, x):
    # first, second and third convolutional layer
    x = self.pool1(torch.relu(self.conv1(x)))
    x = self.pool2(torch.relu(self.conv2(x)))
    x = self.pool3(torch.relu(self.conv3(x)))

    # passing fully connected layers through relu activation layers (except last one)
    x = x.view(-1, 64 * 8 * 8)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
# train and test functions
def train(train, model, loss, optimizer, epochs = 10):
  for epoch in range(epochs):
    epoch_loss = 0.0
    for data in train:
      # getting inputs, labels, feeding forward
      inputs, labels = data
      optimizer.zero_grad()
      outputs = model.forward(inputs)
      
      # calculating loss, optimizer, stepping and adding to epoch loss
      data_loss = loss(outputs, labels)
      loss.backward()
      optimizer.step()
      epoch_loss += data_loss.item()
    print("Epoch loss on epoch " + str(epoch) +":", epoch_loss)
  print("Training complete!")
  return
def test(model, x):
  return model.conv2(x)

# Training

In [ ]:
# creating model
model = CNN()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# training using model
train(train_dataloader, model, loss, optimizer, 10)

FileNotFoundError: ignored

# Testing

< explanation >